In [1]:
import pandas as pd
import numpy as np
import time
import requests


In [2]:
def get_data(s, visited):
    rows = []
    
    ind1 = s.find('Fight History - Pro')
    ind1 = s.find('<tr class="odd">', ind1)
    ind2 = s.find('</table>', ind1)
    
    s = s[ind1:ind2]
    
    ind1 = s.find('final_result')
    
    while ind1 > 0:
        ind1 = s.find('>', ind1) + 1
        ind2 = s.find('<', ind1)
        fight_result = s[ind1:ind2]
        
        keyword = '/fighter/'
        ind1 = s.find(keyword, ind2) + len(keyword)
        ind2 = s.find('"', ind1)
        opponent = s[ind1:ind2]
        
        opp_url = 'https://www.sherdog.com/fighter/{}'.format(opponent)
        if opp_url not in visited:
            keyword = '<span class="sub_line">'
            ind1 = s.find(keyword, ind2) + len(keyword)
            ind2 = s.find('<', ind1)
            fight_date = s[ind1:ind2]
            #fight_date = datetime.datetime.strptime(fight_date, '%b / %d / %Y')

            keyword = '<td>'
            ind1 = s.find(keyword, ind2) + len(keyword)
            ind2 = s.find('<', ind1)
            fight_end_type = s[ind1:ind2]

            keyword = '<td>'
            ind1 = s.find(keyword, ind2) + len(keyword)
            ind2 = s.find('<', ind1)
            fight_rounds = s[ind1:ind2]

            keyword = '<td>'
            ind1 = s.find(keyword, ind2) + len(keyword)
            ind2 = s.find('<', ind1)
            fight_last_round_time = s[ind1:ind2]

            row = [fight_date, opponent, fight_result, fight_end_type, fight_rounds, fight_last_round_time]

            rows.append(row)
        
        ind1 = s.find('final_result', ind2)

    return rows
  

In [4]:
to_visit = ['https://www.sherdog.com/fighter/Conor-McGregor-29688']
visited = []
df_list = []

i = 0
while len(to_visit) > 0:
    url = to_visit.pop(0)
    
    if url not in visited:
        print('Visiting: {}'.format(url))
        
        # get the name of the fighter.
        ind = url.rfind('/')
        fighter_name = url[(ind + 1):]

        # visit website and get name and fight history
        error=0
        response=None
        try:
            response = requests.get(url)
        except Exception as inst:
            error = 1
            print("Error receiving response from server. ",inst)
            raise

        html_txt = response.text

        rows = get_data(html_txt, visited)
        df2 = pd.DataFrame(data=rows, columns=['date', 'fighter1', 'result', 'end_type', 'round', 'end_time'])
        df2['fighter2'] = fighter_name

        df_list.append(df2)
        visited.append(url)

        for row in rows:
            opp = row[1]
            url_to_visit = 'https://www.sherdog.com/fighter/{}'.format(opp)
            to_visit.append(url_to_visit)

        time.sleep(1.2)
        
        i += 1
        
        # do this to stop it early.
        if i >= 10:
            break
        


Visiting: https://www.sherdog.com/fighter/Conor-McGregor-29688
Visiting: https://www.sherdog.com/fighter/Donald-Cerrone-15105
Visiting: https://www.sherdog.com/fighter/Khabib-Nurmagomedov-56035
Visiting: https://www.sherdog.com/fighter/Eddie-Alvarez-9265
Visiting: https://www.sherdog.com/fighter/Nate-Diaz-11451
Visiting: https://www.sherdog.com/fighter/Jose-Aldo-11506
Visiting: https://www.sherdog.com/fighter/Chad-Mendes-38393
Visiting: https://www.sherdog.com/fighter/Dennis-Siver-9817
Visiting: https://www.sherdog.com/fighter/Dustin-Poirier-50529
Visiting: https://www.sherdog.com/fighter/Diego-Brandao-25097


In [5]:
df = pd.concat(df_list)

In [6]:
df

,date,fighter1,result,end_type,round,end_time,fighter2
0,Jan / 18 / 2020,Donald-Cerrone-15105,win,TKO (Head Kick and Punches),1,0:40,Conor-McGregor-29688
1,Oct / 06 / 2018,Khabib-Nurmagomedov-56035,loss,Submission (Neck Crank),4,3:03,Conor-McGregor-29688
2,Nov / 12 / 2016,Eddie-Alvarez-9265,win,TKO (Punches),2,3:04,Conor-McGregor-29688
3,Aug / 20 / 2016,Nate-Diaz-11451,win,Decision (Majority),5,5:00,Conor-McGregor-29688
4,Mar / 05 / 2016,Nate-Diaz-11451,loss,Submission (Rear-Naked Choke),2,4:12,Conor-McGregor-29688
...,...,...,...,...,...,...,...
33,Apr / 21 / 2007,Ronys-Torres-20447,loss,TKO,2,0:00,Diego-Brandao-25097
34,Apr / 01 / 2007,Daniel-Trindade-10729,loss,Submission (Rear Naked Choke),3,2:25,Diego-Brandao-25097
35,Mar / 28 / 2007,Jorge-Dalton-129297,win,TKO (Punches),1,0:00,Diego-Brandao-25097
36,Dec / 16 / 2006,Elifrank-Cariolano-24185,win,TKO (Punches),1,3:41,Diego-Brandao-25097


In [7]:
df.loc[df['date'] == 'Nov / 30 / 00-1', 'date'] = 'Nov / 30 / 2000' # a weird value in a few of the fights.

In [8]:
df['date'] = pd.to_datetime(df['date'], format='%b / %d / %Y') # convert date to datetime object.